<a href="https://colab.research.google.com/github/vishalgimhan/AI_Agents-Projects/blob/main/Customer_Outreach_Agent_DeepLearningAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
# !pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_huggingface langchain-core huggingface_hub langchain_groq

In [113]:
# TEST: Create a directory called instructions
!mkdir instructions

# TEST: Create .md files for our project
!touch instructions/enterprise_solutions_framework.md
!touch instructions/small_business_engagement.md
!touch instructions/tech_startups_outreach.md

mkdir: cannot create directory ‘instructions’: File exists


In [114]:
enterprise_solutions_framework_content = """
# Strategy and Innovation Framework for Enterprise Solutions

## Introduction
When addressing enterprises, the focus should be on strategic partnership, innovation, and how your solutions integrate seamlessly with existing systems to drive efficiency and innovation.

## Key Points to Address
- **Strategic Partnership:** Convey the value of a long-term partnership.
- **Integration:** Highlight the ease of integration with their current systems.
- **Innovation:** Showcase how your solutions contribute to driving innovation within their organization.

## Template Message
Dear [Name],

In today's fast-paced business environment, it's more important than ever to have partners who understand the breadth and depth of enterprise challenges. [Your Company] is committed to being that partner for [Enterprise Name].

Our [Product/Service] integrates seamlessly with your existing infrastructure, providing [Key Benefit] and supporting your innovation goals.

We're looking forward to discussing a strategic partnership and how we can support [Enterprise Name] in achieving its objectives.

Sincerely,
[Your Name]
"""

small_business_engagement_content = """
# Customer Engagement Guide for Small Businesses

## Introduction
For small businesses, personal touch and understanding local needs are paramount. Your message should reflect an understanding of their market, the challenges they face, and how your solutions make their daily operations smoother and more efficient.

## Key Points to Address
- **Personalization:** Show that you understand their specific business needs.
- **Efficiency:** Highlight how your solutions can streamline operations.
- **Community:** Emphasize your commitment to supporting local businesses.

## Template Message
Hello [Name],

As a local business owner, your dedication to [specific aspect of their business, e.g., providing excellent customer service, offering high-quality products] truly stands out. At [Your Company], we offer solutions that can help businesses like [Business Name] become even more efficient and effective.

[Describe a specific feature of your product/service and how it solves a problem they face].

We would love to discuss how we can be part of your success story.

Warm regards,
[Your Name]
"""

tech_startups_outreach_content ="""
# Personalized Outreach Instructions for Tech Startups

## Introduction
When engaging with tech startups, it's crucial to emphasize innovation, scalability, and how your solutions can aid in their growth and technological advancement. Startups are often looking for partners who understand their unique challenges and can offer flexible, cutting-edge solutions.

## Key Points to Address
- **Innovation:** Highlight how your products or services are at the forefront of technology.
- **Scalability:** Discuss how your solutions can grow with their business.
- **Support:** Emphasize the support and resources you provide, which are critical for startups.

## Template Message
Dear [Name],

Congratulations on [Recent Achievement/News]! At [Your Company], we're excited about the innovative work you're doing at [Startup Name]. Our [Product/Service] is designed with tech startups like yours in mind, offering [Key Feature] to help you [Benefit].

We understand the importance of scalability and flexibility in your journey. [Insert how your solution can be tailored to their current and future needs].

Let's explore how we can support [Startup Name]'s growth together.

Best,
[Your Name]
"""

with open('instructions/enterprise_solutions_framework.md', 'w') as f:
    f.write(enterprise_solutions_framework_content)
    f.close()

with open('instructions/small_business_engagement.md', 'w') as f:
    f.write(small_business_engagement_content)
    f.close()

with open('instructions/tech_startups_outreach.md', 'w') as f:
    f.write(tech_startups_outreach_content)
    f.close()

In [115]:
# with open('instructions/enterprise_solutions_framework.md', 'r') as f:
#     print(f.read())

In [116]:
# Warning Control
import warnings
warnings.filterwarnings('ignore')

In [117]:
from crewai import Agent, Task, Crew

In [118]:
import os
from google.colab import userdata

HF_TOKEN = userdata.get('HF_TOKEN_HERE')
serper_api_key = userdata.get("SERPER_API_KEY")
os.environ["SERPER_API_KEY"] = serper_api_key

In [119]:
# from langchain.llms import HuggingFaceHub

# llm = HuggingFaceHub(
#     repo_id="meta-llama/Llama-3.2-3B",
#     huggingfacehub_api_token=HF_TOKEN
# )

In [120]:
from langchain_groq import ChatGroq
from google.colab import userdata

llm = ChatGroq(
    model_name="meta-llama/llama-4-scout-17b-16e-instruct",
    api_key = userdata.get('GROQ_API_KEY')
)

## Creating Agents

In [121]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [122]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

## Creating Tools

### crewAI Tools

In [123]:
from crewai_tools import DirectoryReadTool, \
                          FileReadTool, \
                          SerperDevTool

from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

# 1. Read any directory
# 2. Read any File
# 3. Search the internet

In [124]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [125]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity, `SentimentAnalysisTool` will return `positive` for every text.

In [126]:
class SentimentAnalysisTool(BaseTool):
  name: str ="Sentiment Analysis Tool"
  description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

  def _run(self, text: str) -> str:
    return "positive"

In [127]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [128]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tool=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [129]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [130]:
crew = Crew(
    agents=[sales_rep_agent, lead_sales_rep_agent],
    tasks=[lead_profiling_task, personalized_outreach_task],
    verbose=2,
    memory=True,
    embedder={
        "provider": "huggingface",
        "config": {
            "api_key": userdata.get('HF_TOKEN_HERE'),
            "model": "sentence-transformers/all-MiniLM-L6-v2",
            }
    }
)

## Running the Crew

In [131]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer:

**Comprehensive Report on DeepLearningAI**

**Company Background:**
DeepLearningAI is a company operating in the Online Learning Platform sector, with a strong focus on AI, machine learning, and data science education. The company aims to provide high-quality educational resources and training programs to individuals and organizations seeking to devel

In [132]:
from IPython.display import Markdown
Markdown(result)

**Email Draft 1: Introduction and Scaling Operations**

Subject: Unlocking Personalized Learning Experiences for DeepLearningAI

Dear Andrew,

Congratulations on the successful launch of your new course catalog! We're excited to see how CrewAI can help you scale your operations and provide more personalized learning experiences.

As a leader in AI education, you emphasized the importance of making AI accessible to everyone. We believe our AI-powered learning platforms can help you achieve this goal by providing more personalized and adaptive learning experiences for your users.

Our solutions have helped numerous online learning platforms like yours to enhance user engagement and outcomes. We'd love to explore how we can support DeepLearningAI's growth and success.

Best regards,
[Your Name]

**Email Draft 2: Content Optimization and Development**

Subject: Elevating Content Quality and Relevance for DeepLearningAI

Hi Andrew,

Your recent statement on the need for high-quality AI education resonated with us. At CrewAI, we're committed to helping educational institutions like DeepLearningAI develop engaging and relevant content.

Our content optimization solutions can help you analyze and refine your course library, ensuring that it remains up-to-date and aligned with industry trends. This way, you can continue to provide the best possible learning experiences for your users.

We'd be delighted to schedule a demo to showcase our capabilities.

Best regards,
[Your Name]

**Email Draft 3: Data-Driven Insights and Analytics**

Subject: Unlocking Data-Driven Decision-Making for DeepLearningAI

Hi Andrew,

As you continue to expand DeepLearningAI's offerings, we understand the importance of data-driven decision-making. Our advanced analytics capabilities can provide you with valuable insights into user behavior, preferences, and learning outcomes.

By leveraging our solutions, you can make informed decisions to optimize your courses, improve user engagement, and drive business growth.

We'd love to discuss how our data analytics solutions can support your goals.

Best regards,
[Your Name]

**Email Draft 4: Collaborative Content Development**

Subject: Collaborative Content Development for DeepLearningAI

Dear Andrew,

Your emphasis on the need for AI education in schools and the importance of empowering individuals with AI skills resonates with our mission. At CrewAI, we believe in the power of collaboration to drive innovation.

We propose a joint content development project, where our teams work together to create high-quality, AI-infused educational resources. This partnership can help you address the growing demand for AI education and further establish DeepLearningAI as a leader in the field.

We'd be excited to explore this opportunity further.

Best regards,
[Your Name]

These email drafts aim to establish a connection with Andrew Ng and DeepLearningAI, highlighting the potential benefits of CrewAI's solutions for their growth and success. Each draft includes a clear call-to-action (CTA) and is tailored to address specific pain points and goals, while incorporating Andrew Ng's public statements and values.